In [1]:
import pandas as pd
import warnings
from IPython.utils import io
import sys
import numpy as np
from functools import reduce

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

stars_dir = '~/GitHub/stars-data-builder/'
hos_dir = '~/Desktop/Rush/CMS_HospitalArchives/'

dates_df = pd.DataFrame(columns = ['Measure ID', 'Start Date', 'End Date'])

In [2]:
def curate(df):

    try:
        df = df[df['PROVIDER_ID'] != np.nan]
        df['PROVIDER_ID'] = df['PROVIDER_ID'].values.astype(str)
        
        ids = df['PROVIDER_ID'].tolist()
        ids2 = []
        for i in ids:
            if len(i) < 6:
                i = '0' + i
            ids2.append(i)
        df['PROVIDER_ID'] = ids2
        
    except:
        pass
    
    for c in list(df):    
        try:
            df[c] = df[c].str.replace("\t","")
        except:
            pass

    if 'Unnamed: 0' in list(df):
        df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
    return df

# Use 2023 SAS input file to get the column labels needed for 2025 prognostication

Remove OP-2 and OP-3B, as these will no longer be reported in Care Compare

In [3]:
sas_input_df_2023 = pd.read_sas(stars_dir + 'Reproduce_Stars_Input/2023/Input_File/alldata_2023jul.sas7bdat', 
                           format = 'sas7bdat', encoding = "utf8")


sas_input_df_2023.drop(labels=['OP_2', 'OP_3B', 'OP_2_DEN', 'OP_3B_DEN'], axis=1, inplace=True)
sas_cols_2023 = list(sas_input_df_2023)

print(len(sas_cols_2023), 'columns in 2023 Stars SAS input file (not all get used by the SAS programs):')
print(sas_cols_2023, '\n')

print('Create columns for 2024 data')

sas_cols = list(sas_cols_2023)

sas_input_df_2023 = curate(sas_input_df_2023)
prvdrs_2023 = sas_input_df_2023['PROVIDER_ID'].unique().tolist()

print(len(prvdrs_2023), 'hospitals in 2023 Stars output file\n')
prvdrs_2023 = sorted(list(set(prvdrs_2023)))

print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')

sas_input_df_2023.head()

91 columns in 2023 Stars SAS input file (not all get used by the SAS programs):
['PROVIDER_ID', 'HAI_1_DEN_VOL', 'HAI_2_DEN_VOL', 'HAI_3_DEN_VOL', 'HAI_4_DEN_VOL', 'HAI_5_DEN_VOL', 'HAI_6_DEN_VOL', 'HAI_1_DEN_PRED', 'HAI_2_DEN_PRED', 'HAI_3_DEN_PRED', 'HAI_4_DEN_PRED', 'HAI_5_DEN_PRED', 'HAI_6_DEN_PRED', 'HAI_1', 'HAI_2', 'HAI_3', 'HAI_4', 'HAI_5', 'HAI_6', 'READM_30_HOSP_WIDE', 'READM_30_HIP_KNEE', 'EDAC_30_HF', 'READM_30_COPD', 'EDAC_30_AMI', 'EDAC_30_PN', 'MORT_30_STK', 'MORT_30_PN', 'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'READM_30_HOSP_WIDE_DEN', 'READM_30_HIP_KNEE_DEN', 'EDAC_30_HF_DEN', 'READM_30_COPD_DEN', 'EDAC_30_AMI_DEN', 'EDAC_30_PN_DEN', 'MORT_30_STK_DEN', 'MORT_30_PN_DEN', 'MORT_30_HF_DEN', 'MORT_30_COPD_DEN', 'MORT_30_AMI_DEN', 'COMP_HIP_KNEE_DEN', 'OP_8', 'OP_8_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'PSI_4_SURG_COMP', 'PSI_4_SURG_COMP_DEN', 'PSI_90_

,PROVIDER_ID,HAI_1_DEN_VOL,HAI_2_DEN_VOL,HAI_3_DEN_VOL,HAI_4_DEN_VOL,HAI_5_DEN_VOL,HAI_6_DEN_VOL,HAI_1_DEN_PRED,HAI_2_DEN_PRED,HAI_3_DEN_PRED,HAI_4_DEN_PRED,HAI_5_DEN_PRED,HAI_6_DEN_PRED,HAI_1,HAI_2,HAI_3,HAI_4,HAI_5,HAI_6,READM_30_HOSP_WIDE,READM_30_HIP_KNEE,EDAC_30_HF,READM_30_COPD,EDAC_30_AMI,EDAC_30_PN,MORT_30_STK,MORT_30_PN,MORT_30_HF,MORT_30_COPD,MORT_30_AMI,COMP_HIP_KNEE,READM_30_HOSP_WIDE_DEN,READM_30_HIP_KNEE_DEN,EDAC_30_HF_DEN,READM_30_COPD_DEN,EDAC_30_AMI_DEN,EDAC_30_PN_DEN,MORT_30_STK_DEN,MORT_30_PN_DEN,MORT_30_HF_DEN,MORT_30_COPD_DEN,MORT_30_AMI_DEN,COMP_HIP_KNEE_DEN,OP_8,OP_8_DEN,OP_10,OP_10_DEN,OP_13,OP_13_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,PSI_90_SAFETY,IMM_3_DEN,IMM_3,HCP_COVID_19_DEN,HCP_COVID_19,PC_01,PC_01_DEN,SEP_1,SEP_1_DEN,H_RESP_RATE_P,H_COMP_1_STAR_RATING,H_COMP_2_STAR_RATING,H_COMP_3_STAR_RATING,H_COMP_5_STAR_RATING,H_COMP_6_STAR_RATING,H_COMP_7_STAR_RATING,H_GLOB_STAR_RATING,H_INDI_STAR_RATING,H_NUMB_COMP,PSI_90_SAFETY_DEN,MORT_30_CABG,MORT_30_CABG_DEN,READM_30_CABG,READM_30_CABG_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN
0,010001,10024.0,17731.0,154.0,200.0,101908.0,101451.0,10.597,26.630,4.548,1.845,9.412,72.686,0.661,0.300,1.099,0.0,0.850,0.660,0.142,0.042,21.8,0.199,1.9,-1.5,0.164,0.159,0.083,0.085,0.124,0.024,3058.0,98.0,755.0,202.0,319.0,436.0,489.0,407.0,630.0,182.0,317.0,102.0,0.425,146.0,0.057,1488.0,0.067,208.0,205.0,323.0,0.03,51079.0,NaN,NaN,0.81,16.0,173.39,120.0,1.01,3795.0,0.97,2323.0,0.737,0.09,34.0,0.46,146.0,15.0,2.0,3.0,2.0,4.0,4.0,3.0,3.0,3.5,434.0,2046.895485,0.047,172.0,0.117,165.0,14.1,254.0,10.2,214.0,4.7,214.0,1.0,688.0
1,010005,3713.0,8670.0,88.0,NaN,38413.0,35686.0,2.450,4.995,2.512,NaN,1.999,10.484,3.673,1.201,1.194,NaN,0.000,0.858,0.139,0.040,9.3,0.176,4.7,2.2,0.166,0.218,0.169,0.081,0.126,0.018,1258.0,178.0,157.0,234.0,38.0,361.0,100.0,369.0,153.0,195.0,52.0,134.0,0.545,191.0,0.138,1214.0,0.043,208.0,146.0,1003.0,0.03,54503.0,0.73,15.0,0.99,108.0,142.88,35.0,0.91,2593.0,0.90,2026.0,0.821,0.01,194.0,0.59,242.0,16.0,3.0,4.0,2.0,3.0,3.0,3.0,3.0,3.5,717.0,819.043002,NaN,NaN,NaN,NaN,14.6,850.0,11.1,108.0,5.7,108.0,0.9,362.0
2,010006,7318.0,11755.0,91.0,NaN,62709.0,54159.0,7.924,15.296,2.523,NaN,4.164,22.618,0.757,0.196,0.396,NaN,1.441,0.088,0.142,0.048,-2.3,0.177,25.9,42.3,0.189,0.178,0.122,0.078,0.165,0.034,2555.0,246.0,550.0,235.0,312.0,538.0,261.0,528.0,468.0,209.0,295.0,234.0,0.412,97.0,0.110,1168.0,0.014,217.0,144.0,363.0,0.01,41137.0,0.57,14.0,0.88,75.0,157.42,84.0,1.10,2292.0,0.64,2694.0,0.651,0.00,37.0,0.58,142.0,17.0,2.0,3.0,1.0,2.0,3.0,2.0,2.0,2.5,1358.0,1487.163359,0.035,117.0,0.156,109.0,12.5,1505.0,NaN,NaN,NaN,NaN,1.1,468.0
3,010007,NaN,NaN,NaN,NaN,NaN,5413.0,NaN,NaN,NaN,NaN,NaN,2.148,NaN,NaN,NaN,NaN,NaN,0.466,0.151,NaN,36.3,0.197,NaN,-12.6,NaN,0.217,0.139,0.103,NaN,NaN,272.0,NaN,51.0,72.0,NaN,99.0,NaN,106.0,45.0,63.0,NaN,NaN,NaN,NaN,0.059,169.0,NaN,NaN,119.0,1202.0,0.03,11120.0,NaN,NaN,0.63,68.0,NaN,NaN,0.99,318.0,0.61,277.0,0.574,NaN,NaN,0.93,55.0,23.0,3.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,173.0,142.073902,NaN,NaN,NaN,NaN,15.3,118.0,NaN,NaN,NaN,NaN,1.0,56.0
4,010008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.145,NaN,NaN,NaN,NaN,NaN,NaN,0.197,NaN,NaN,NaN,NaN,93.0,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,0.021,97.0,NaN,NaN,113.0,346.0,0.00,6205.0,NaN,NaN,0.52,23.0,NaN,NaN,NaN,125.0,0.46,169.0,0.623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.3,62.0,NaN,NaN,NaN,NaN,NaN,NaN


## HAIs

In [4]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_04_2024/Healthcare_Associated_Infections-Hospital.csv')
#print(df['Measure ID'].unique())

measures = ['HAI_1_ELIGCASES', 'HAI_1_DOPC', 'HAI_1_SIR', 'HAI_2_ELIGCASES', 'HAI_2_DOPC', 'HAI_2_SIR', 
            'HAI_3_ELIGCASES', 'HAI_3_DOPC', 'HAI_3_SIR', 'HAI_4_ELIGCASES', 'HAI_4_DOPC', 'HAI_4_SIR', 
            'HAI_5_ELIGCASES', 'HAI_5_DOPC', 'HAI_5_SIR', 'HAI_6_ELIGCASES', 'HAI_6_DOPC', 'HAI_6_SIR']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0)

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score'], axis=1)

hai_df = pd.DataFrame(columns=['Facility ID']) 
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    hai_df = hai_df.merge(tdf2, on='Facility ID', how='outer')
    
hai_df.rename(columns={'HAI_1_ELIGCASES': 'HAI_1_DEN_PRED',
                       'HAI_1_DOPC': 'HAI_1_DEN_VOL',
                       'HAI_1_SIR': 'HAI_1',
                       'HAI_2_ELIGCASES': 'HAI_2_DEN_PRED',
                       'HAI_2_DOPC': 'HAI_2_DEN_VOL',
                       'HAI_2_SIR': 'HAI_2',
                       'HAI_3_ELIGCASES': 'HAI_3_DEN_PRED',
                       'HAI_3_DOPC': 'HAI_3_DEN_VOL',
                       'HAI_3_SIR': 'HAI_3',
                       'HAI_4_ELIGCASES': 'HAI_4_DEN_PRED',
                       'HAI_4_DOPC': 'HAI_4_DEN_VOL',
                       'HAI_4_SIR': 'HAI_4',
                       'HAI_5_ELIGCASES': 'HAI_5_DEN_PRED',
                       'HAI_5_DOPC': 'HAI_5_DEN_VOL',
                       'HAI_5_SIR': 'HAI_5',
                       'HAI_6_ELIGCASES': 'HAI_6_DEN_PRED',
                       'HAI_6_DOPC': 'HAI_6_DEN_VOL',
                       'HAI_6_SIR': 'HAI_6',
                       'Facility ID': 'PROVIDER_ID',
                   }, inplace=True)

for c in list(hai_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass

print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')
hai_df = curate(hai_df)

72 remaining features: ['COMP_HIP_KNEE', 'COMP_HIP_KNEE_DEN', 'EDAC_30_AMI', 'EDAC_30_AMI_DEN', 'EDAC_30_HF', 'EDAC_30_HF_DEN', 'EDAC_30_PN', 'EDAC_30_PN_DEN', 'HCP_COVID_19', 'HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'IMM_3', 'IMM_3_DEN', 'MORT_30_AMI', 'MORT_30_AMI_DEN', 'MORT_30_CABG', 'MORT_30_CABG_DEN', 'MORT_30_COPD', 'MORT_30_COPD_DEN', 'MORT_30_HF', 'MORT_30_HF_DEN', 'MORT_30_PN', 'MORT_30_PN_DEN', 'MORT_30_STK', 'MORT_30_STK_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_32', 'OP_32_DEN', 'OP_35_ADM', 'OP_35_ADM_DEN', 'OP_35_ED', 'OP_35_ED_DEN', 'OP_36', 'OP_36_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN', 'PSI_4_SURG_COMP', 'PSI_4_SURG_COMP_DEN', 'PSI_90_SAFETY', 'PSI_90_SAFETY_DEN', 

## Unplanned Hospital Visits


In [5]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_04_2024/Unplanned_Hospital_Visits-Hospital.csv')
measures = ['EDAC_30_AMI', 'EDAC_30_HF', 'EDAC_30_PN', 'OP_32', 'OP_35_ADM', 'OP_35_ED', 'OP_36', 
            'READM_30_CABG', 'READM_30_COPD', 'READM_30_HIP_KNEE', 'READM_30_HOSP_WIDE']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0)

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Denominator', 'Measure ID', 'Score'], axis=1)

uhv_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    tdf2[m + '_DEN'] = tdf1['Denominator'].tolist()
    uhv_df = uhv_df.merge(tdf2, on='Facility ID', how='outer')

uhv_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)

for c in list(uhv_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass
    
print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')
uhv_df = curate(uhv_df)

50 remaining features: ['COMP_HIP_KNEE', 'COMP_HIP_KNEE_DEN', 'HCP_COVID_19', 'HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'IMM_3', 'IMM_3_DEN', 'MORT_30_AMI', 'MORT_30_AMI_DEN', 'MORT_30_CABG', 'MORT_30_CABG_DEN', 'MORT_30_COPD', 'MORT_30_COPD_DEN', 'MORT_30_HF', 'MORT_30_HF_DEN', 'MORT_30_PN', 'MORT_30_PN_DEN', 'MORT_30_STK', 'MORT_30_STK_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN', 'PSI_4_SURG_COMP', 'PSI_4_SURG_COMP_DEN', 'PSI_90_SAFETY', 'PSI_90_SAFETY_DEN', 'SEP_1', 'SEP_1_DEN'] 



## COMPLICATIONS AND DEATHS

In [6]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_04_2024/Complications_and_Deaths-Hospital.csv')

measures = ['MORT_30_AMI', 'MORT_30_CABG', 'MORT_30_COPD', 'MORT_30_HF', 
            'MORT_30_PN', 'MORT_30_STK', 'PSI_04', 'COMP_HIP_KNEE',
            'PSI_90']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score', 'Denominator'], axis=1)

cad_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    tdf2[m + '_DEN'] = tdf1['Denominator'].tolist()
    cad_df = cad_df.merge(tdf2, on='Facility ID', how='outer')
    
cad_df.rename(columns={'Facility ID': 'PROVIDER_ID',
                       'PSI_04': 'PSI_4_SURG_COMP',
                       'PSI_04_DEN': 'PSI_4_SURG_COMP_DEN',
                       'PSI_90': 'PSI_90_SAFETY',
                       'PSI_90_DEN': 'PSI_90_SAFETY_DEN',
                   }, inplace=True)

for c in list(cad_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass
    
print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')
cad_df = curate(cad_df)

32 remaining features: ['HCP_COVID_19', 'HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'IMM_3', 'IMM_3_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN', 'SEP_1', 'SEP_1_DEN'] 



## TIMELY AND EFFECTIVE CARE

Everything except PC-01, which for 2023 is located in the Maternal Health files of the hospitals data archive

In [7]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_04_2024/Timely_and_Effective_Care-Hospital.csv')

measures = ['IMM_3', 'OP_18b', 'OP_22', 'OP_23', 'OP_29', 'SEP_1', 'HCP_COVID_19']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Sample', 'Measure ID', 'Score'], axis=1)

tec_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    if m == 'HCP_COVID_19':
        pass
    else:
        tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    
    tec_df = tec_df.merge(tdf2, on='Facility ID', how='outer')
    
tec_df.rename(columns={'Facility ID': 'PROVIDER_ID', 
                       'OP_18b': 'OP_18B',
                       'OP_18b_DEN': 'OP_18B_DEN',
                      }, inplace=True)

for c in list(tec_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass

print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')
tec_df = curate(tec_df)


19 remaining features: ['HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN'] 



In [8]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_04_2024/VA_TE.csv')

measures = ['IMM-3', 'OP-18b', 'OP-22', 'OP-23', 'OP-29', 'SEP-1', 'HCP-COVID-19']

tec3_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    
    tec3_df = tec3_df.merge(tdf2, on='Facility ID', how='outer')
    
tec3_df.rename(columns={'Facility ID': 'PROVIDER_ID', 
                        'OP-18b': 'OP_18B',
                        'OP-18b_DEN': 'OP_18B_DEN',
                        
                        'IMM-3': 'IMM_3', 
                        'IMM-3_DEN': 'IMM_3_DEN', 
                        
                        'OP-22': 'OP_22', 
                        'OP-22_DEN': 'OP_22_DEN', 
                        
                        'OP-23': 'OP_23', 
                        'OP-23_DEN': 'OP_23_DEN', 
                        
                        'OP-29': 'OP_29', 
                        'OP-29_DEN': 'OP_29_DEN', 
                        
                        'SEP-1': 'SEP_1', 
                        'SEP-1_DEN': 'SEP_1_DEN', 
                        
                        'HCP-COVID-19': 'HCP_COVID_19',
                        'HCP-COVID-19_DEN': 'HCP_COVID_19_DEN',
                        
                      }, inplace=True)

tec3_df.replace('Not Available', np.nan, inplace=True)
print(tec3_df.shape)
tec3_df = curate(tec3_df)
print(tec3_df.shape)

print(tec_df.shape)
print(tec3_df.shape)
tec_df = pd.concat([tec_df, tec3_df])
print(tec_df.shape)

tec_df.head()

(137, 15)
(137, 15)
(4677, 14)
(137, 15)
(4814, 15)


,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,SEP_1,SEP_1_DEN,HCP_COVID_19,HCP_COVID_19_DEN
0,010001,95,3905,215,350,5,52960,Not Available,Not Available,47,17,66,127,80.7,NaN
1,010005,80,2700,147,1082,3,56820,Not Available,Not Available,96,180,74,251,79.8,NaN
2,010006,67,2536,177,361,1,42286,69,16,85,82,56,139,79,NaN
3,010007,53,350,130,1239,4,11202,Not Available,Not Available,23,111,86,36,57.9,NaN
4,010008,45,126,118,336,0,6239,Not Available,Not Available,67,24,Not Available,Not Available,81.2,NaN


## TIMELY AND EFFECTIVE CARE

Only PC-01, which for 2023 is located in the Maternal Health files of the hospitals data archive

In [9]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_04_2024/Maternal_Health-Hospital.csv')
measures = ['PC_01']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score', 'Sample'], axis=1)

tec2_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    tec2_df = tec2_df.merge(tdf2, on='Facility ID', how='outer')
    
tec2_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)

for c in list(tec2_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass

print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')

tec2_df = curate(tec2_df)


17 remaining features: ['HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_8', 'OP_8_DEN'] 



## HCAHPS

In [10]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_04_2024/HCAHPS-Hospital.csv')

measures = ['H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 
            'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_CLEAN_STAR_RATING',  'H_QUIET_STAR_RATING', 
            'H_RECMND_STAR_RATING', 'H_HSP_RATING_STAR_RATING']

tdf = df[df['HCAHPS Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf.rename(columns={'HCAHPS Measure ID': 'Measure ID'}, inplace=True)
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['HCAHPS Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'HCAHPS Measure ID', 'Patient Survey Star Rating', 
                        'Number of Completed Surveys', 'Survey Response Rate Percent'], axis=1)

HCAHPS_df = pd.DataFrame(columns=['Facility ID'])
for i, m in enumerate(measures):
    tdf1 = df[df['HCAHPS Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Patient Survey Star Rating'].tolist()
    if i == 0:
        tdf2['H_NUMB_COMP'] = tdf1['Number of Completed Surveys'].tolist()
        tdf2['H_RESP_RATE_P'] = tdf1['Survey Response Rate Percent'].tolist()
        
    HCAHPS_df = HCAHPS_df.merge(tdf2, on='Facility ID', how='outer')
    

HCAHPS_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)
HCAHPS_df['H_HSP_RATING_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_HSP_RATING_STAR_RATING'] = HCAHPS_df['H_HSP_RATING_STAR_RATING'].astype(int)
HCAHPS_df['H_RECMND_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_RECMND_STAR_RATING'] = HCAHPS_df['H_RECMND_STAR_RATING'].astype(int)
HCAHPS_df['H_GLOB_STAR_RATING'] = np.round((HCAHPS_df['H_HSP_RATING_STAR_RATING'] + HCAHPS_df['H_RECMND_STAR_RATING']) / 2, 1)
HCAHPS_df['H_GLOB_STAR_RATING'].replace(0, np.nan, inplace=True)

HCAHPS_df['H_CLEAN_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_CLEAN_STAR_RATING'] = HCAHPS_df['H_CLEAN_STAR_RATING'].astype(int)
HCAHPS_df['H_QUIET_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_QUIET_STAR_RATING'] = HCAHPS_df['H_QUIET_STAR_RATING'].astype(int)
HCAHPS_df['H_INDI_STAR_RATING'] = np.round((HCAHPS_df['H_CLEAN_STAR_RATING'] + HCAHPS_df['H_QUIET_STAR_RATING']) / 2, 1)
HCAHPS_df['H_INDI_STAR_RATING'].replace(0, np.nan, inplace=True)

HCAHPS_df.drop(labels = ['H_CLEAN_STAR_RATING',  'H_QUIET_STAR_RATING', 'H_RECMND_STAR_RATING', 'H_HSP_RATING_STAR_RATING'], axis=1, inplace=True)

for c in list(HCAHPS_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass

print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')
HCAHPS_df = curate(HCAHPS_df)

7 remaining features: ['HCP_COVID_19_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_8', 'OP_8_DEN'] 



## Outpatient Imaging Efficiency

The July 2023 Overall Star Ratings were calculated using the measure data from the January 2023 Care Compare update, with the re-released OP-13 measure data publicly reported in April 2023 on Care Compare.

Make two dataframes: 1 for OP-13 (April 2023) and 1 for other measures (Jan 2023)

In [11]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_04_2024/Outpatient_Imaging_Efficiency-Hospital.csv')
measures = ['OP-13', 'OP-8', 'OP-10']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score'], axis=1)

oie_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    oie_df = oie_df.merge(tdf2, on='Facility ID', how='outer')
    
    
oie_df.rename(columns={'Facility ID': 'PROVIDER_ID',
                       'OP-13': 'OP_13',
                       'OP-8': 'OP_8',
                       'OP-10': 'OP_10',
                   }, inplace=True)

for c in list(oie_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass

print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')
oie_df1 = curate(oie_df)


4 remaining features: ['HCP_COVID_19_DEN', 'OP_10_DEN', 'OP_13_DEN', 'OP_8_DEN'] 



## MERGE DATAFRAME AND COMPARE TO SAS FILE

In [12]:
print(dates_df.shape)
dates_df.to_csv(stars_dir + "2024/04_2024_measure_dates.csv", index=False)

(59, 3)


In [13]:
main_df = tec_df.merge(tec2_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(cad_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(HCAHPS_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(uhv_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(hai_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(oie_df1, on='PROVIDER_ID', how='outer')

main_df['OP_10_DEN'] = np.nan
main_df['OP_13_DEN'] = np.nan
main_df['OP_8_DEN'] = np.nan
main_df['HCP_COVID_19_DEN'] = np.nan

for c in list(main_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass

print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')
print(main_df.shape)
main_df.head()

0 remaining features: [] 

(4814, 91)


,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,SEP_1,SEP_1_DEN,HCP_COVID_19,HCP_COVID_19_DEN,PC_01,PC_01_DEN,MORT_30_AMI,MORT_30_AMI_DEN,MORT_30_CABG,MORT_30_CABG_DEN,MORT_30_COPD,MORT_30_COPD_DEN,MORT_30_HF,MORT_30_HF_DEN,MORT_30_PN,MORT_30_PN_DEN,MORT_30_STK,MORT_30_STK_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,COMP_HIP_KNEE,COMP_HIP_KNEE_DEN,PSI_90_SAFETY,PSI_90_SAFETY_DEN,H_COMP_1_STAR_RATING,H_NUMB_COMP,H_RESP_RATE_P,H_COMP_2_STAR_RATING,H_COMP_3_STAR_RATING,H_COMP_5_STAR_RATING,H_COMP_6_STAR_RATING,H_COMP_7_STAR_RATING,H_GLOB_STAR_RATING,H_INDI_STAR_RATING,EDAC_30_AMI,EDAC_30_AMI_DEN,EDAC_30_HF,EDAC_30_HF_DEN,EDAC_30_PN,EDAC_30_PN_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN,READM_30_CABG,READM_30_CABG_DEN,READM_30_COPD,READM_30_COPD_DEN,READM_30_HIP_KNEE,READM_30_HIP_KNEE_DEN,READM_30_HOSP_WIDE,READM_30_HOSP_WIDE_DEN,HAI_1_DEN_PRED,HAI_1_DEN_VOL,HAI_1,HAI_2_DEN_PRED,HAI_2_DEN_VOL,HAI_2,HAI_3_DEN_PRED,HAI_3_DEN_VOL,HAI_3,HAI_4_DEN_PRED,HAI_4_DEN_VOL,HAI_4,HAI_5_DEN_PRED,HAI_5_DEN_VOL,HAI_5,HAI_6_DEN_PRED,HAI_6_DEN_VOL,HAI_6,OP_13,OP_8,OP_10,OP_10_DEN,OP_13_DEN,OP_8_DEN
0,010001,95,3905,215,350,5,52960,Not Available,Not Available,47,17,66,127,80.7,NaN,0,35,12,278,4.1,132,8.8,107,8.9,549,18,400,14.8,398,184.68,134,2.7,49,1.21,Not Applicable,3,599,16,4,3,3,4,4,4.0,3.5,-15.4,274,23.4,614,23.6,403,12.9,170,11.9,202,4.9,202,1.1,668,10.5,126,19,117,3.8,49,14.2,2912,9.388,8925,0.852,24.092,16886,0.291,5.656,206,1.414,0.713,76,Not Available,11.334,104126,0.971,63.972,104126,0.391,2.8,38,6.1,NaN,NaN,NaN
1,010005,80,2700,147,1082,3,56820,Not Available,Not Available,96,180,74,251,79.8,NaN,1,194,13.6,27,Not Available,Not Available,9.9,126,14.9,121,23.3,289,15.3,81,183.49,43,2.3,155,0.97,Not Applicable,3,760,17,4,2,3,3,3,3.0,3.0,Not Available,Not Available,22.1,129,-6.2,285,14.2,739,7.9,107,5.5,107,1.9,406,Not Available,Not Available,16.6,136,3.4,172,12.8,1052,1.921,3155,1.041,3.718,8120,1.345,2.555,93,0.783,0.350,36,Not Available,1.871,36620,1.603,9.977,34919,0.401,4.2,47.7,12,NaN,NaN,NaN
2,010006,67,2536,177,361,1,42286,69,16,85,82,56,139,79,NaN,4,25,16.5,254,3.6,95,9.9,148,12.5,388,19.5,469,17.2,227,173.63,96,4.6,145,1.17,Not Applicable,2,1603,19,3,2,2,3,2,2.0,2.5,28.1,273,-4.7,441,-0.4,472,12.1,1355,Not Available,Not Available,Not Available,Not Available,1.4,484,12.4,89,17.6,158,5.3,138,13.4,2310,5.720,5258,0.175,11.266,8820,0.178,3.157,119,0.000,0.380,44,Not Available,5.293,66994,1.323,31.456,63233,0.064,4.5,46.2,10.1,NaN,NaN,NaN
3,010007,53,350,130,1239,4,11202,Not Available,Not Available,23,111,86,36,57.9,NaN,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,13.7,34,12.5,26,28.5,88,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,0.95,Not Applicable,3,182,24,5,4,3,4,4,4.0,3.0,Not Available,Not Available,-1.9,31,-9.4,72,13.4,109,Not Available,Not Available,Not Available,Not Available,1.2,59,Not Available,Not Available,20,34,4.2,26,15.7,258,0.203,330,Not Available,0.663,1214,Not Available,0.050,2,Not Available,Not Available,Not Available,Not Available,0.127,5462,Not Available,2.307,5462,1.300,Not Available,Not Available,3.4,NaN,NaN,NaN
4,010008,45,126,118,336,0,6239,Not Available,Not Available,67,24,Not Available,Not Available,81.2,NaN,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Applicable,Not Available,51,31,Not Available,Not Available,Not Available,Not Available,Not Available,NaN,NaN,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,12.9,42,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,14.8,69,0.026,47,N

In [14]:
ls = np.setdiff1d(list(main_df), sas_cols)
print(ls)

main_df = main_df.filter(items=sas_cols, axis=1)
print(main_df.shape)

main_df.head()

[]
(4814, 91)


,PROVIDER_ID,HAI_1_DEN_VOL,HAI_2_DEN_VOL,HAI_3_DEN_VOL,HAI_4_DEN_VOL,HAI_5_DEN_VOL,HAI_6_DEN_VOL,HAI_1_DEN_PRED,HAI_2_DEN_PRED,HAI_3_DEN_PRED,HAI_4_DEN_PRED,HAI_5_DEN_PRED,HAI_6_DEN_PRED,HAI_1,HAI_2,HAI_3,HAI_4,HAI_5,HAI_6,READM_30_HOSP_WIDE,READM_30_HIP_KNEE,EDAC_30_HF,READM_30_COPD,EDAC_30_AMI,EDAC_30_PN,MORT_30_STK,MORT_30_PN,MORT_30_HF,MORT_30_COPD,MORT_30_AMI,COMP_HIP_KNEE,READM_30_HOSP_WIDE_DEN,READM_30_HIP_KNEE_DEN,EDAC_30_HF_DEN,READM_30_COPD_DEN,EDAC_30_AMI_DEN,EDAC_30_PN_DEN,MORT_30_STK_DEN,MORT_30_PN_DEN,MORT_30_HF_DEN,MORT_30_COPD_DEN,MORT_30_AMI_DEN,COMP_HIP_KNEE_DEN,OP_8,OP_8_DEN,OP_10,OP_10_DEN,OP_13,OP_13_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,PSI_90_SAFETY,IMM_3_DEN,IMM_3,HCP_COVID_19_DEN,HCP_COVID_19,PC_01,PC_01_DEN,SEP_1,SEP_1_DEN,H_RESP_RATE_P,H_COMP_1_STAR_RATING,H_COMP_2_STAR_RATING,H_COMP_3_STAR_RATING,H_COMP_5_STAR_RATING,H_COMP_6_STAR_RATING,H_COMP_7_STAR_RATING,H_GLOB_STAR_RATING,H_INDI_STAR_RATING,H_NUMB_COMP,PSI_90_SAFETY_DEN,MORT_30_CABG,MORT_30_CABG_DEN,READM_30_CABG,READM_30_CABG_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN
0,010001,8925,16886,206,76,104126,104126,9.388,24.092,5.656,0.713,11.334,63.972,0.852,0.291,1.414,Not Available,0.971,0.391,14.2,3.8,23.4,19,-15.4,23.6,14.8,18,8.9,8.8,12,2.7,2912,49,614,117,274,403,398,400,549,107,278,49,38,NaN,6.1,NaN,2.8,NaN,215,350,5,52960,Not Available,Not Available,47,17,184.68,134,1.21,3905,95,NaN,80.7,0,35,66,127,16,3,4,3,3,4,4,4.0,3.5,599,Not Applicable,4.1,132,10.5,126,12.9,170,11.9,202,4.9,202,1.1,668
1,010005,3155,8120,93,36,36620,34919,1.921,3.718,2.555,0.350,1.871,9.977,1.041,1.345,0.783,Not Available,1.603,0.401,12.8,3.4,22.1,16.6,Not Available,-6.2,15.3,23.3,14.9,9.9,13.6,2.3,1052,172,129,136,Not Available,285,81,289,121,126,27,155,47.7,NaN,12,NaN,4.2,NaN,147,1082,3,56820,Not Available,Not Available,96,180,183.49,43,0.97,2700,80,NaN,79.8,1,194,74,251,17,3,4,2,3,3,3,3.0,3.0,760,Not Applicable,Not Available,Not Available,Not Available,Not Available,14.2,739,7.9,107,5.5,107,1.9,406
2,010006,5258,8820,119,44,66994,63233,5.720,11.266,3.157,0.380,5.293,31.456,0.175,0.178,0.000,Not Available,1.323,0.064,13.4,5.3,-4.7,17.6,28.1,-0.4,17.2,19.5,12.5,9.9,16.5,4.6,2310,138,441,158,273,472,227,469,388,148,254,145,46.2,NaN,10.1,NaN,4.5,NaN,177,361,1,42286,69,16,85,82,173.63,96,1.17,2536,67,NaN,79,4,25,56,139,19,2,3,2,2,3,2,2.0,2.5,1603,Not Applicable,3.6,95,12.4,89,12.1,1355,Not Available,Not Available,Not Available,Not Available,1.4,484
3,010007,330,1214,2,Not Available,5462,5462,0.203,0.663,0.050,Not Available,0.127,2.307,Not Available,Not Available,Not Available,Not Available,Not Available,1.300,15.7,4.2,-1.9,20,Not Available,-9.4,Not Available,28.5,12.5,13.7,Not Available,Not Available,258,26,31,34,Not Available,72,Not Available,88,26,34,Not Available,Not Available,Not Available,NaN,3.4,NaN,Not Available,NaN,130,1239,4,11202,Not Available,Not Available,23,111,Not Available,Not Available,0.95,350,53,NaN,57.9,Not Available,Not Available,86,36,24,3,5,4,3,4,4,4.0,3.0,182,Not Applicable,Not Available,Not Available,Not Available,Not Available,13.4,109,Not Available,Not Available,Not Available,Not Available,1.2,59
4,010008,47,494,Not Available,Not Available,2889,1918,0.026,0.269,Not Available,Not Available,0.067,0.352,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,14.8,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,69,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,NaN,0,NaN,Not Available,NaN,118,336,0,6239,Not Available,Not Available,67,24,Not Available,Not Available,Not Available,126,45,NaN,81.2,Not Available,Not Available,Not Available,Not Available,31,Not Available,Not Available,Not Available,Not

In [15]:
prvdrs_main1 = sorted(main_df['PROVIDER_ID'].tolist())

ls = ['READM_30_HIP_KNEE', 'READM_30_COPD', 'MORT_30_STK', 'MORT_30_PN',
      'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'OP_22',
      'OP_23', 'OP_29', 'IMM_3', 'PC_01', 'SEP_1', 'MORT_30_CABG',
      'READM_30_CABG', 'READM_30_HOSP_WIDE', 'OP_8',
      'OP_10', 'OP_13', 'HCP_COVID_19']

for col in list(main_df):
    if col != 'PROVIDER_ID':
        main_df[col] = pd.to_numeric(main_df[col], errors='coerce')
        
for l in ls: 
    main_df[l] = main_df[l] * 0.01


In [16]:
prvdrs = []
for p in main_df['PROVIDER_ID'].tolist():
    if 'F' in p:
        p = p[:-1]
        p = p + '666666'
    prvdrs.append(p)

main_df['PROVIDER_ID'] = prvdrs

main_df['PROVIDER_ID'] = pd.to_numeric(main_df['PROVIDER_ID'], errors='coerce')
main_df.sort_values(by=['PROVIDER_ID'], ascending = True, inplace = True)
main_df.to_csv(stars_dir + "Reproduce_Stars_Input/2025/Input_File/data_for_2025_prognostications_from_April2024.csv", 
               index=False)
